# Test Cache in Jupyter Notebook

In [10]:
import joblib

print(joblib)

<module 'joblib' from '/Users/saggese/.conda/envs/p1_develop/lib/python3.7/site-packages/joblib/__init__.py'>


In [1]:
import helpers.dbg as dbg
import helpers.cache as hcac

# Define computation function

This function will be subjected to cache.

In [57]:
def computation_function(a, b):
    out = a + b
    print("Sum: %s + %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 3

computation_function(*inputs)

Sum: 1 + 2 = 3


3

In [52]:
def computation_function(a, b):
    # hello
    #assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

computation_function(*inputs)

Multiplication: 1 * 2 = 2


2

## LRU Cache

In [111]:
def hello():
    print("hello")
    
print(id(hello))

140612447448096


In [112]:
import functools
mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

hello


In [113]:
mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

hello


In [105]:
def hello():
    print("hello")
    
print(id(hello))

140612447445648


In [106]:
def hello():
    assert 0
    print("good bye")
    
print(id(hello))

140612447446512


In [96]:
#hello()
print(id(hello))
mem_func()

140612447446656


AssertionError: 

In [95]:
mem_func = functools.lru_cache(maxsize=None)(hello)

mem_func()

AssertionError: 

## Joblib cache

In [194]:
import joblib

disk_cache = joblib.Memory("./joblib.cache", verbose=0, compress=1)
disk_cache.clear()

In [195]:
def hello():
    print("hello")
print(id(hello))
    
cached_hello = disk_cache.cache(hello)
    
print(id(cached_hello))

140612475429520
140612476055120


In [196]:
cached_hello()

hello


In [197]:
cached_hello()

In [198]:
def hello():
    print("hello")

In [199]:
cached_hello()

In [200]:
def hello():
    print("good bye")
   
cached_hello = disk_cache.cache(hello)

In [201]:
cached_hello()

good bye


In [202]:
def hello():
    print("good bye")
   
cached_hello = disk_cache.cache(hello)

cached_hello()

## Memory cache

In [61]:
def computation_function(a, b):
    out = a + b
    print("Sum: %s + %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 3

In [62]:
memory_cached_computation = hcac.Cached(computation_function, use_mem_cache=True, use_disk_cache=False)

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

print("memory caching checks passed")

Skipping disk cache


Sum: 1 + 2 = 3
memory caching checks passed


In [63]:
def computation_function(a, b):
    # hello
    #assert 0
    out = a * b
    print("Multiplication: %s * %s = %s" % (a, b, out))
    return out


inputs = (1, 2)
exp_output = 2

dbg.dassert_eq(memory_cached_computation(*inputs), exp_output)
print(memory_cached_computation.get_last_cache_accessed())
dbg.dassert_eq(memory_cached_computation.get_last_cache_accessed(), "mem")

AssertionError: 
################################################################################
* Failed assertion *
'3'
==
'2'
################################################################################


In [17]:
memory_cached_computation._func.__code__
memory_cached_computation._func.co_filename


AttributeError: 'function' object has no attribute 'co_filename'

In [18]:
computation_function.__code__

<code object computation_function at 0x7fe2e1e5adb0, file "<ipython-input-3-9007637ed568>", line 1>

## Disk cache

In [30]:
disk_cached_computation = hcac.Cached(computation_function, use_mem_cache=False, use_disk_cache=True)

disk_cached_computation.clear_disk_cache()

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

print("disk caching checks passed")

computation_function: clearing disk cache
Skipping memory cache
Skipping memory cache


Multiplication: 1 * 2 = 2
disk caching checks passed


In [50]:
print(disk_cached_computation._execute_func_from_disk_cache.func)

from joblib.func_inspect import get_func_code, get_func_name, filter_args
      
print(get_func_code(disk_cached_computation._execute_func_from_disk_cache.func)[0])
dbg.dassert_eq(disk_cached_computation(*inputs), exp_output)
dbg.dassert_eq(disk_cached_computation.get_last_cache_accessed(), "disk")

Skipping memory cache


<function Cached.__init__.<locals>._execute_func_from_disk_cache at 0x7fe2e2f91710>
        @self._disk_cache.cache
        def _execute_func_from_disk_cache(*args: Any, **kwargs: Any) -> Any:
            # If we get here, we didn't hit neither memory nor the disk cache.
            self._last_used_disk_cache = False
            _LOG.debug(
                "%s(args=%s kwargs=%s): execute the intrinsic function",
                self._func.__name__,
                args,
                kwargs,
            )
            obj = self._func(*args, **kwargs)
            return obj



## Full cache

In [6]:
fully_cached_computation = hcac.Cached(computation_function, use_mem_cache=True, use_disk_cache=True)

fully_cached_computation.clear_disk_cache()

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "no_cache")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("Clear mem cache")
fully_cached_computation.clear_memory_cache()

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "disk")

dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

print("full caching checks passed")

computation_function: clearing disk cache
computation_function: clearing memory cache


Multiplication: 1 * 2 = 2
Clear mem cache
full caching checks passed


In [7]:
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

In [8]:
# This should fail all the times, because we clear the memory cache.
fully_cached_computation.clear_memory_cache()
dbg.dassert_eq(fully_cached_computation(*inputs), exp_output)
dbg.dassert_eq(fully_cached_computation.get_last_cache_accessed(), "mem")

computation_function: clearing memory cache


AssertionError: 
################################################################################
* Failed assertion *
'disk'
==
'mem'
################################################################################
